In [1]:
import filetype
from pathlib import Path
import nbimporter
import gradio as gr

In [2]:
def get_gradio_launcher(func):
    view = gr.Interface(
        fn=func,
        inputs=[gr.Textbox(label="Your message:"),gr.File(label="Upload file"),gr.Dropdown(["GPT", "GEMINI"], label="Select model")],
        outputs=[gr.Markdown(label="Response:")],
        flagging_mode="never"
    )
    return view
    #view.launch()#view.launch(server_name="10.75.32.62", server_port=7866)

In [3]:
def is_ascii(file_path):
    try:
        with open(file_path, 'rb') as f:
            content = f.read()
            content.decode('ascii')  # Will raise UnicodeDecodeError if not ASCII
        return True
    except UnicodeDecodeError:
        return False
        
def get_file_path_str(file_path):
    if(isinstance(file_path,Path)):
        file_path=file_path.as_posix()
    return file_path 
    
def detect_mime_type(file_path):
    file_path=get_file_path_str(file_path)
    if is_ascii(file_path):
        return "text/plain"

    kind = filetype.guess(file_path)
    if kind:
        return kind.mime    

    return None

# Example usage
print(detect_mime_type("C:/Projects/llm_engg/llm_engineering/requirements.txt"))


text/plain


In [4]:
def getFileContent(fp):  
    fp=get_file_path_str(fp)
    with open(fp, 'r') as file:
        file_content = file.read()
        return file_content
    return ""

In [5]:
def append_conv(conv, msg):
    conv.append(msg)
    return conv

In [6]:
def create_gpt_prompt(qt, role):
    """
    Creates a correctly formatted Python dictionary for an OpenAI API message.
    """
    return {
        "role": role,
        "content": qt
    }

In [7]:
def create_gpt_tool_response_dict(tool, response):
    return {
        "tool_call_id" : tool.id,
        "role" : "tool",
        "name" : tool.function.name,
        "content" : response
    }

In [8]:
def reset_conv_history(conv,sys_msg):
    conv=[]
    conv.append(sys_msg)
    return conv